# Set up

In [97]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!cp /content/drive/MyDrive/MMDS/spark-3.1.1-bin-hadoop3.2.tgz .
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [98]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [99]:
import findspark
findspark.init()

# Main libraries

In [100]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import rand, array
import pyspark.sql.functions as F
from pyspark.sql.types import *
from collections import defaultdict
import numpy as np
from sklearn.cluster import AgglomerativeClustering

# Hyperparameters

In [101]:
N_REPS = 4
N_CLUSTERS = 6
SHRINK_FACTOR = 0.2

# Test

In [102]:
spark = SparkSession.builder.appName("Experiment").getOrCreate()

In [103]:
import os

path = "/content/drive/MyDrive/MMDS/Endterm/Question2/embedding.json"

if os.path.exists(path):
    print("Path exists!")
else:
    print("Path does not exist.")

Path exists!


In [104]:
df = spark.read.json(path)


In [105]:
df.show(5)

+--------------------+-------+
|           embedding|user_id|
+--------------------+-------+
|[-16.796118656868...|     31|
|[-0.1880540258735...|     65|
|[-2.3890451791994...|     53|
|[-1.9212580037324...|     34|
|[-4.3522241197278...|     28|
+--------------------+-------+
only showing top 5 rows



In [106]:
df.count()

74

# Sampling

In [107]:
# Sampling
sampled_df = df.sample(False, 0.25, seed=42)

In [108]:
sampled_df.show(5, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
|embedding                                                                                                                                                                                                                                                                                                                                 

In [109]:
sampled_df.count()

16

# Hierarchical Clustering

In [110]:
sampled_in_memory = sampled_df.select("embedding").collect()

In [111]:
sampled_in_memory[0]

Row(embedding=[-0.9598459091748469, -0.4246291302869712, -0.2285888389534422, 0.5723830268209384, 0.5015473155640188, -0.05279022990555792, 0.5991510264788718, -0.287799982630296, 0.7378180024968382, -0.1400883369709, 0.3693544793122994, -0.7546414463099685, -0.8623152549132151, -0.658938014223161, 0.15336394660881772, -0.2517572305873124, 0.4447933498644217, 1.1578775686891658, -0.17149433760129545, 0.015462913727149996, 0.09733935865940342, -0.365464526157242, 0.2460744965053182, -0.8494279148455302, -0.7711592367808227, -0.253529833015817, 0.46533980515229445, 0.5908457435712421, -0.18138738814989108, 0.545771074659318, 0.41496085161499335, -0.8383231720125232])

In [112]:
np_sampled = np.array([np.array(row["embedding"])
                        for row in sampled_in_memory])

In [113]:
np_sampled.shape

(16, 32)

In [114]:
clustering = AgglomerativeClustering(n_clusters=N_CLUSTERS, linkage="ward")
model = clustering.fit(np_sampled)

In [115]:
predicted = model.labels_

In [179]:
predicted

array([0, 0, 3, 0, 0, 4, 5, 0, 1, 0, 0, 0, 0, 2, 0, 0])

In [116]:
n = len(predicted)
cluster_id_A_points = defaultdict(list)
for i in range(n):
    cluster_id_A_points[predicted[i]].append(np_sampled[i].tolist())

In [117]:
list(cluster_id_A_points.items())[0]

(0,
 [[-0.9598459091748469,
   -0.4246291302869712,
   -0.2285888389534422,
   0.5723830268209384,
   0.5015473155640188,
   -0.05279022990555792,
   0.5991510264788718,
   -0.287799982630296,
   0.7378180024968382,
   -0.1400883369709,
   0.3693544793122994,
   -0.7546414463099685,
   -0.8623152549132151,
   -0.658938014223161,
   0.15336394660881772,
   -0.2517572305873124,
   0.4447933498644217,
   1.1578775686891658,
   -0.17149433760129545,
   0.015462913727149996,
   0.09733935865940342,
   -0.365464526157242,
   0.2460744965053182,
   -0.8494279148455302,
   -0.7711592367808227,
   -0.253529833015817,
   0.46533980515229445,
   0.5908457435712421,
   -0.18138738814989108,
   0.545771074659318,
   0.41496085161499335,
   -0.8383231720125232],
  [-6.534820613289457,
   -3.6619022702724373,
   0.23286289624818818,
   3.45424605393703,
   2.374468197577119,
   -1.1716347673719754,
   1.64238181839149,
   -0.45807660299423975,
   0.052089587112749476,
   -2.181543161345548,
   0.6977

# Choose representatives

In [118]:
for cluster_id, points in list(cluster_id_A_points.items()):
    print(cluster_id, len(points))

0 11
3 1
4 1
5 1
1 1
2 1


In [119]:
def choose_reps(centroid, points, num_reps):
    if len(points) <= num_reps:
        return points
    reps = [centroid]

    for _ in range(num_reps):
        point_A_minimum_distances = []
        for point in points:

            if point in reps:
                continue


            point_A_minimum_distances.append(
                (
                    point,
                    min([np.linalg.norm(
                    np.array(rep) - np.array(point)) for rep in reps])
                )
            )

        rep, distance = max(point_A_minimum_distances, key=lambda x: x[1])
        reps.append(rep)

    return reps[1:]

# Shriking representatives

In [120]:
def shrink_reps(representatives, centroid, shrink_factor):

    shrunken_reps = [
        [
            rep_coords + shrink_factor * (rep_coords - centroid_coords)
            for rep_coords, centroid_coords in zip(rep, centroid)
        ]
        for rep in representatives
    ]

    return shrunken_reps

In [121]:
def compute_centroid(points):
    return [
        sum(point[i] for point in points) / len(points)
        for i in range(len(points[0]))
    ]

In [122]:
processed_clusters = []
for cluster_id, points in list(cluster_id_A_points.items()):

    # Compute centroid
    centroid = compute_centroid(points)

    # Choose representatives (reps)
    reps = choose_reps(centroid, points, N_REPS)
    # # Shrinking reps
    shrunken_reps = shrink_reps(reps, centroid, SHRINK_FACTOR)

    processed_clusters.append({
        "cluster_id": cluster_id,
        "centroid": centroid,
        "points": points,
        "reps": shrunken_reps,
    })


In [123]:
np.linalg.norm(
    np.array(processed_clusters[0]["points"][0]) - np.array(processed_clusters[1]["points"][0]))

24.984093862256746

# Merging clusters by their reps

In [147]:
def merge_close_clusters(clusters, threshold=100):

    # Initialize a parent-child mapping
    merge_map = defaultdict(list)

    for i, cluster1 in enumerate(clusters):
        for j, cluster2 in enumerate(clusters[i+1:], start=i+1):
            # Check if the clusters are close

            if any(
                np.linalg.norm(np.array(rep1) - np.array(rep2)) <= threshold
                for rep1 in cluster1['reps']
                for rep2 in cluster2['reps']
            ):

                merge_map[cluster1['cluster_id']].append(cluster2['cluster_id'])

    # Convert merge_map into list of tuples (merged groups)
    merged_groups = []
    visited = set()

    def gather_group(cluster_id):
        # Recursively gather all children of a cluster.
        if cluster_id in visited:
            return []
        visited.add(cluster_id)
        group = [cluster_id]
        for child in merge_map.get(cluster_id, []):
            group.extend(gather_group(child))
        return group

    # List of tuples, each tuple contains merge_cluster_id s
    for cluster_id in merge_map:
        if cluster_id not in visited:
            merged_groups.append(tuple(gather_group(cluster_id)))


    # Combine clusters in merged_groups
    merged_clusters = []
    merged_ids = set()

    id = 0
    for group in merged_groups:
        merged_ids.update(group)
        new_reps = []
        for cluster_id in group:
            new_reps.extend(clusters[cluster_id]['reps'])

        merged_clusters.append({
            'cluster_id': id,
            'reps': new_reps
        })
        id += 1

    unmerged_clusters = []

    for cluster in clusters:
        if cluster['cluster_id'] not in merged_ids:
            unmerged_clusters.append({"cluster_id": id,
                                      "reps": cluster["reps"]})
            id += 1

    return merged_clusters + unmerged_clusters

In [172]:
new_clusters = merge_close_clusters(processed_clusters, threshold=2)

In [173]:
for _ in new_clusters:
    cluster_id, reps = _.values()
    print(cluster_id, len(reps))

0 4
1 1
2 1
3 1
4 1
5 1


In [174]:
def assign_point_to_cluster_id(point, clusters):
    cluster_id_A_min_dist = []
    for cluster in clusters:

        rep, min_dist = min([
                    (
                        rep,
                        np.linalg.norm(np.array(point) - np.array(rep))
                    )
                    for rep in cluster["reps"]], key=lambda x: x[1])

        cluster_id_A_min_dist.append((cluster["cluster_id"], rep, min_dist))

    cluster_id, rep, min_dist = min(cluster_id_A_min_dist, key=lambda x: x[2])
    return cluster_id, rep

In [175]:
assign_point_to_cluster_id_udf = F.udf(
    lambda point: assign_point_to_cluster_id(point, new_clusters),
    StructType([
        StructField("cluster_id", IntegerType()),
        StructField("rep", ArrayType(DoubleType()))
    ])
)

In [176]:
result_df = df.withColumn("tmp",
        assign_point_to_cluster_id_udf(F.col("embedding"))) \
        .select("tmp.cluster_id", "tmp.rep", F.col("embedding").alias("point"))

In [177]:
result_df.filter(F.col("cluster_id")==0).count()

64

In [178]:
result_df.count()

74